In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
path = Path('data/midi')

In [5]:
import pandas as pd

In [6]:
source_dir = 'midi_sources'
out_dir = 'midi_transform_v2'
source_csv = path/'metadata'/f'{source_dir}.csv'
out_csv = path/out_dir/f'{out_dir}.csv'

In [7]:
df = pd.read_csv(source_csv, index_col=0); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,14,15,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_offset,ht_time_signature,inferred_key,...,instruments,midi,midi_title,parts,seconds,section,song_url,source,time_signature,title
0,0,wayne-sharpe,128.0,NaN,128.0,C,1.0,0.0,4.0,C major,...,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,yu-gi-oh3,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song
1,1,weezer,NaN,NaN,128.0,C,NaN,0.0,4.0,NaN,...,NaN,data/midi/midi_sources/hooktheory/pianoroll/w/...,My New Song,intro-and-verse,NaN,intro-and-verse,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,NaN,beverly-hills
2,2,weezer,108.0,NaN,108.0,Eb,1.0,-3.0,4.0,E- major,...,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,falling for you intro,"intro,verse,chorus,solo",11.111111,intro,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,falling-for-you
3,3,weezer,121.0,"Pop,Rock",121.0,Ab,1.0,4.0,4.0,A- major,...,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,Buddy Holly,"verse,pre-chorus,chorus,bridge,solo",43.388430,solo,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,buddy-holly
4,4,wavves,180.0,NaN,180.0,A,1.0,3.0,4.0,B minor,...,"Piano,Piano",data/midi/midi_sources/hooktheory/pianoroll/w/...,dreams of grandeur,verse,21.333333,verse,https://www.hooktheory.com/theorytab/view/wavv...,hooktheory,4/4,dreams-of-grandeur


In [8]:
df_filtered = df.loc[df['time_signature'] == '4/4']; df_filtered.shape

(30201, 21)

In [9]:
from data_sources import process_parallel

In [10]:
from midi_transform import *

### Need better midi formatting

In [11]:
def transform_midi_from_row(idxrow):
    idx,row = idxrow
    midi_file = row.midi
    
    if not isinstance(midi_file, str) or not Path(midi_file).exists(): return idx,None
    
    out_file = Path(midi_file.replace(f'/{source_dir}/', f'/{out_dir}/'))
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    
    offset = row.ht_offset if row.source == 'hooktheory' else row.inferred_offset
    try:
        out_file = transform_midi(midi_file, out_file, offset=offset)
    except Exception as e:
        print('Exception with midifile:', midi_file, e)
        return idx, None
    return idx, str(out_file)

In [12]:
# for r in df_filtered.iterrows():
#     transform_midi_from_row(r)

In [13]:
idx2out = process_parallel(transform_midi_from_row, df_filtered.iterrows(), total=df_filtered.shape[0])

Exception with midifile: data/midi/midi_sources/freemidi/genre-dance-eletric/Kylie Minogue - Celebration.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/freemidi/genre-dance-eletric/Scooter - Hyper Hyper.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/freemidi/genre-dance-eletric/Calvin Harris - I Need Your Love.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/freemidi/genre-dance-eletric/Tatu - Ya soshla s uma (karaoke).mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/freemidi/genre-dance-eletric/Robbie Williams - Stalkers Day Off.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/freemidi/genre-dance-eletric/Madonna - 4 Minutes.mid list index out of range
Exception with midifile: data/midi/midi_sources/freemidi/genre-dance-eletric/Daft Punk - The Son Of Flyn.mid 'NoneType' ob

Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Roxette - Almost Unreal.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Elton John - Lady Samantha.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Tina Turner - Simply The Best.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Elton John - Chloe.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Bette Midler - Beaches.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Leann Rimes - Don't Ever Lose That Love In Your Eyes.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Alison Moyet - All Cried Out.mid 'NoneType' object has no attribute 'write'
Exception with midifi

Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Nintendo_-_Pokemon_Liquid_Crystal_Violet_City_Piano_Cover.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Warren_G_-_Regulate.mid list index out of range
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Animenz_-_This_game_by_Animenz_in_C_piano.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Nintendo_-_Pkmn-GSC-Route-1-Theme-Music-Box-2-Times.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_1_Rainbow_Route_Piano_Cover_Play_2_Times.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_9_Candy_Constellation_Play_2_Times.mid Error: info channel is not channel 0
Exception with 

Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Geri_Halliwell_-_Look_At_Me.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/TV_Themes_-_Party_of_Five.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Lesley_Gore_-_Its_My_Party.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Jim_Croce_-_Dont_Mess_Around_With_Jim.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Tavares_-_It_Only_Takes_A_Minute.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Disney_Themes_-_Chim_Chim_Cheree.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Mariah Carey - So Cold.mid 'NoneType' object has no 

Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Animenz_-_This_Game_in_C.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Animenz_-_This_Game_in_F.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Foreigner_-_Waiting_for_a_Girl_Like_You.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Nintendo_-_Pokemon_FRLG_Battle_Gym_Leader_Theme_Guitar_Cover_2_Times.mid Error: info channel is not channel 0
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Dave_Mathews_-_Two_Step.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Rush_-_Tom_Sawyer.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/The_Corrs_-_What_Can_I_Do_Remix.mid 'NoneType' objec

Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Laura_Brannigan_-_Gloria.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Guns_n_Roses_-_Knockin_On_Heaven's_Door.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/freemidi/genre-dance-eletric/Four Tops - If I Were A Carpenter, If I Had A Hammer.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Snoop_Dogg_-_Ain't_Nothing_Personal.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/freemidi/genre-pop/Pet Shop Boys - Go West.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Christmas_Carols_-_Santa_Claus_Is_Coming_To_Town.mid 'NoneType' object has no attribute 'write'
Exception with midifile: data/midi/midi_sources/midiworld/named_midi/Bi

In [14]:
tdf = pd.DataFrame(data={out_dir: list(idx2out.values())}, index=list(idx2out.keys()))

In [15]:
merged_df = df.join(tdf)

In [16]:
tdf.shape, df.shape, merged_df.shape

((30201, 1), (33746, 21), (33746, 22))

In [17]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,index,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_offset,ht_time_signature,inferred_key,...,midi,midi_title,parts,seconds,section,song_url,source,time_signature,title,midi_transform_v2
0,0,wayne-sharpe,128.0,NaN,128.0,C,1.0,0.0,4.0,C major,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,yu-gi-oh3,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song,data/midi/midi_transform_v2/hooktheory/pianoro...
1,1,weezer,NaN,NaN,128.0,C,NaN,0.0,4.0,NaN,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,My New Song,intro-and-verse,NaN,intro-and-verse,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,NaN,beverly-hills,NaN
2,2,weezer,108.0,NaN,108.0,Eb,1.0,-3.0,4.0,E- major,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,falling for you intro,"intro,verse,chorus,solo",11.111111,intro,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,falling-for-you,data/midi/midi_transform_v2/hooktheory/pianoro...
3,3,weezer,121.0,"Pop,Rock",121.0,Ab,1.0,4.0,4.0,A- major,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,Buddy Holly,"verse,pre-chorus,chorus,bridge,solo",43.388430,solo,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,buddy-holly,data/midi/midi_transform_v2/hooktheory/pianoro...
4,4,wavves,180.0,NaN,180.0,A,1.0,3.0,4.0,B minor,...,data/midi/midi_sources/hooktheory/pianoroll/w/...,dreams of grandeur,verse,21.333333,verse,https://www.hooktheory.com/theorytab/view/wavv...,hooktheory,4/4,dreams-of-grandeur,data/midi/midi_transform_v2/hooktheory/pianoro...
